In [1]:
import os
import importlib
import torch

from voc12 import my_dataloader
from torch.utils.data import DataLoader
import numpy as np
from utility.image_util import *
from misc import torchutils
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
cam_network = "net.resnet50_cam"
rgbd_cam_weights_name = "../sess/voc_sess/resnet50_rgbd_cam"

val_list = "voc12/val.txt"
voc12_root = "../Dataset/VOC2012/"
depth_root = "../result/depth_img/"

device = "cuda"

In [3]:
model = getattr(importlib.import_module(cam_network), 'Net')(rgbd=True)
# model.load_state_dict(torch.load(rgbd_cam_weights_name + '.pth'), strict=True)
model.train()

In [4]:
val_dataset = my_dataloader.VOC12_DepthClassificationDataset(val_list, voc12_root=voc12_root,
                                                                             depth_root=depth_root,
                                                                             resize_long=(320, 640), hor_flip=True,
                                                                             crop_size=512, crop_method='random')

In [10]:
data_loader = DataLoader(val_dataset)

param_groups = model.trainable_parameters()
optimizer = torchutils.PolyOptimizer([
        {'params': param_groups[0], 'lr': 0.05, 'weight_decay': 1e-4},
        {'params': param_groups[1], 'lr': 0.5, 'weight_decay': 1e-4},
    ], lr=0.05, weight_decay=1e-4, max_step=100)

model = model.to(device)

In [12]:
for step, pack in enumerate(tqdm(data_loader)):

    img = pack['img'].to(device)
    label = pack['label'].to(device)
    x = model(img)
    loss1 = F.multilabel_soft_margin_loss(x, label)
    
    # avg_meter.add({'loss1': loss1.item()})

    optimizer.zero_grad()
    loss1.backward()
    optimizer.step()
    print(loss1)

  0%|          | 3/2913 [00:00<05:41,  8.53it/s]

tensor(0.6941, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6926, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6907, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6890, device='cuda:0', grad_fn=<MeanBackward0>)


  0%|          | 8/2913 [00:00<03:14, 14.90it/s]

tensor(0.6893, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6866, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6871, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.6837, device='cuda:0', grad_fn=<MeanBackward0>)


  0%|          | 14/2913 [00:00<02:37, 18.45it/s]

...

 16%|█▌        | 453/2913 [00:22<02:03, 19.95it/s]

tensor(0.3078, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3749, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.2831, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.2914, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3284, device='cuda:0', grad_fn=<MeanBackward0>)


 16%|█▌        | 458/2913 [00:22<02:01, 20.18it/s]

tensor(0.3523, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3036, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.3038, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.2853, device='cuda:0', grad_fn=<MeanBackward0>)


KeyboardInterrupt: 